In [1]:
from os import walk
import pandas as pd
import numpy as np
import math

In [2]:
path = "..\\results"

In [3]:
def get_paths_folders(path, files):
    d = []
    for (dirpath, dirnames, filenames) in walk(path):
        if files:
            d.extend(filenames[0:10])
        else:
            d.extend(dirnames)
        break
    return d

In [4]:
def slice_dataframe(data):
    UniqueNames = data.organism.unique()

    #create a data frame dictionary to store your data frames
    DataFrameDict = {elem : pd.DataFrame for elem in UniqueNames}
    for key in DataFrameDict.keys():
        DataFrameDict[key] = data[data.organism == key][:].reset_index().drop('index', axis=1)
        
    return DataFrameDict

In [5]:
d = get_paths_folders(path, False)
results_classifiers = {}
for dirname in d: #\results\j48 (if not threshold)   
    if dirname.__contains__("threshold"):
        results_classifiers[dirname] = {}
        newPath = path + "\\" + dirname      
        d1 = get_paths_folders(newPath, False)
        for dirname1 in d1:
            results_classifiers[dirname][dirname1] = {}
            newPath2 = newPath + "\\" + dirname1
            f = get_paths_folders(newPath2, True)
            for filename in f:        
                df = pd.read_csv(newPath2 + "\\" + filename)
                df.drop(df.filter(regex="Unnamed"), axis=1, inplace=True)   
                df = df.replace(",", ".", regex=True)
                df = df.astype({"GMean": float, "ratioReduction": float})
                DataFrameDict = slice_dataframe(df)
                for key in DataFrameDict.keys():           
                    k = 0                   
                    for value in DataFrameDict[key]["GMean"]: 
                        if value <= 0:                           
                            value = value * (-1)
                        DataFrameDict[key].at[k, "GMean"] = value
                        k = k + 1                 
                    DataFrameDict[key] =  DataFrameDict[key].sort_values(by='Fold', ascending=True)

                
                for key in DataFrameDict.keys():
                    if key not in results_classifiers[dirname][dirname1].keys():
                        results_classifiers[dirname][dirname1][key] = list()                   
                    results_classifiers[dirname][dirname1][key].append(DataFrameDict[key])

    else:
        results_classifiers[dirname] = {}
        newPath = path + "\\" + dirname
        f = get_paths_folders(newPath, True)
        for filename in f:        
            df = pd.read_csv(newPath + "\\" + filename)
            df.drop(df.filter(regex="Unnamed"), axis=1, inplace=True)   
            df = df.replace(",", ".", regex=True)
            df = df.astype({"GMean": float, "ratioReduction": float})
            DataFrameDict = slice_dataframe(df)
            for key in DataFrameDict.keys():           
                k = 0
                for value in DataFrameDict[key]["GMean"]:             
                    if value <= 0:
                        value = value * (-1)
                    DataFrameDict[key].at[k, "GMean"] = value
                    k = k + 1            
                DataFrameDict[key] =  DataFrameDict[key].sort_values(by='Fold', ascending=True)
                 
            for key in DataFrameDict.keys():
                if key not in results_classifiers[dirname].keys():
                    results_classifiers[dirname][key] = list()                   
                results_classifiers[dirname][key].append(DataFrameDict[key])  


In [7]:
print(results_classifiers['threshold-5']['J48']['worm'][0])

  organism  populationSize  mutationProbability  crossoverProbability  Fold  \
0     worm             NaN                  NaN                   NaN     0   
1     worm             NaN                  NaN                   NaN     1   
2     worm             NaN                  NaN                   NaN     2   
3     worm             NaN                  NaN                   NaN     3   
4     worm             NaN                  NaN                   NaN     4   
5     worm             NaN                  NaN                   NaN     5   
6     worm             NaN                  NaN                   NaN     6   
7     worm             NaN                  NaN                   NaN     7   
8     worm             NaN                  NaN                   NaN     8   
9     worm             NaN                  NaN                   NaN     9   

   GMean  Best GMean AGMO  ratioReduction  
0  43.55              NaN             NaN  
1  30.51              NaN             NaN 

In [8]:
def make_average(threshold: str = None, classifier: str = None, organism: str = None, agmoOrNot: int = 1, gMean = None, ratioRed = None):
    factorTrunc = 10.0 ** 3
    factorTruncStd = 10.0 ** 1
    if threshold:
        if gMean:
            return (math.trunc((results_classifiers[threshold][classifier][organism][int(agmoOrNot)][gMean].mean() * factorTrunc))/factorTrunc, math.trunc((results_classifiers[threshold][classifier][organism][int(agmoOrNot)][gMean].std() * factorTruncStd))/factorTruncStd)
        else:
            return (math.trunc((results_classifiers[threshold][classifier][organism][int(agmoOrNot)][ratioRed].mean() * 100 * factorTrunc))/factorTrunc, math.trunc((results_classifiers[threshold][classifier][organism][int(agmoOrNot)][ratioRed].std() * 100 * factorTruncStd))/factorTruncStd)
    else:
        if gMean:
            
            return (math.trunc((results_classifiers[classifier][organism][int(agmoOrNot)][gMean].mean() * factorTrunc))/factorTrunc, math.trunc((results_classifiers[classifier][organism][int(agmoOrNot)][gMean].std() * factorTruncStd))/factorTruncStd)
        else:
            return (math.trunc((results_classifiers[classifier][organism][int(agmoOrNot)][ratioRed].mean() * 100 * factorTrunc))/factorTrunc, math.trunc((results_classifiers[classifier][organism][int(agmoOrNot)][ratioRed].std() * 100 * factorTruncStd))/factorTruncStd)


In [14]:
columns = {}
# table_1 = pd.DataFrame(
#     columns= {'Normal', 'T(3)', 'T(4)', 'T(5)', 'T(6)', 'T(7)', 'T(8)', 'T(9)', 'T(10)'}
# )
# table_1.groupby('Normal').aggregate('GMean', 'STD')
#mux = pd.MultiIndex.from_product([['Normal', 'T(3)', 'T(4)', 'T(5)', 'T(6)', 'T(7)', 'T(8)', 'T(9)', 'T(10)'], ['Average','Ratio Reduction']])
mux = pd.MultiIndex.from_product([['Normal', 'T(3)', 'T(4)', 'T(5)', 'T(6)', 'T(7)', 'T(8)', 'T(9)', 'T(10)'], ['Average']])

In [16]:
#sem ratioReduction
table_1_KNN_without_ratioReduction = pd.DataFrame([
        [
            make_average(None, classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-3', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-4', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-5', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-6', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-7', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-8', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-9', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-10', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
        ], 
        [
            make_average(None, classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-3', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-4', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-5', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-6', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-7', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-8', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-9', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-10', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
        ],
        [
            make_average(None, classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-3', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-4', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-5', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-6', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-7', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-8', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-9', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-10', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
        ],
        [
            make_average(None, classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-3', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-4', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-5', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-6', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-7', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-8', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-9', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-10', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
        ],
    ],
    columns=mux, 
    index=['fly', 'mouse', 'worm', 'yeast']
)

table_1_KNN_without_ratioReduction

,Normal,T(3),T(4),T(5),T(6),T(7),T(8),T(9),T(10)
,Average,Average,Average,Average,Average,Average,Average,Average,Average
fly,"(57.339, 11.1)","(57.629, 22.2)","(54.696, 11.2)","(41.288, 19.3)","(35.857, 27.3)","(41.883, 27.4)","(40.577, 23.6)","(36.129, 21.5)","(32.913, 18.6)"
mouse,"(56.013, 12.4)","(55.284, 12.9)","(52.84, 8.5)","(38.545, 22.0)","(32.257, 23.5)","(42.879, 17.1)","(33.947, 24.5)","(38.345, 21.2)","(37.007, 21.5)"
worm,"(58.665, 7.1)","(54.43, 4.4)","(51.208, 5.7)","(49.785, 5.1)","(47.433, 7.2)","(46.389, 9.1)","(44.715, 9.2)","(44.49, 11.8)","(40.468, 9.4)"
yeast,"(42.612, 30.6)","(39.85, 35.6)","(43.308, 26.3)","(48.926, 21.3)","(43.34, 32.0)","(40.205, 29.8)","(37.096, 27.2)","(26.581, 29.1)","(22.109, 29.5)"


In [17]:
#sem ratioReduction
table_1_NB_without_ratioReduction = pd.DataFrame([
        [
            make_average(None, classifier = "NB", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-3', classifier = "NB", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-4', classifier = "NB", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-5', classifier = "NB", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-6', classifier = "NB", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-7', classifier = "NB", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-8', classifier = "NB", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-9', classifier = "NB", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-10', classifier = "NB", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
        ], 
        [
            make_average(None, classifier = "NB", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-3', classifier = "NB", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-4', classifier = "NB", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-5', classifier = "NB", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-6', classifier = "NB", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-7', classifier = "NB", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-8', classifier = "NB", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-9', classifier = "NB", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-10', classifier = "NB", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
        ],
        [
            make_average(None, classifier = "NB", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-3', classifier = "NB", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-4', classifier = "NB", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-5', classifier = "NB", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-6', classifier = "NB", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-7', classifier = "NB", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-8', classifier = "NB", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-9', classifier = "NB", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-10', classifier = "NB", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
        ],
        [
            make_average(None, classifier = "NB", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-3', classifier = "NB", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-4', classifier = "NB", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-5', classifier = "NB", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-6', classifier = "NB", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-7', classifier = "NB", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-8', classifier = "NB", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-9', classifier = "NB", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-10', classifier = "NB", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
        ],
    ],
    columns=mux, 
    index=['fly', 'mouse', 'worm', 'yeast']
)

table_1_NB_without_ratioReduction

,Normal,T(3),T(4),T(5),T(6),T(7),T(8),T(9),T(10)
,Average,Average,Average,Average,Average,Average,Average,Average,Average
fly,"(52.529, 21.3)","(55.013, 7.5)","(46.779, 20.6)","(43.669, 19.7)","(37.699, 19.0)","(37.618, 18.1)","(31.815, 17.9)","(31.815, 17.9)","(10.318, 16.6)"
mouse,"(46.503, 26.5)","(47.295, 18.5)","(41.779, 16.8)","(30.827, 28.7)","(27.601, 24.6)","(30.98, 27.5)","(28.269, 25.3)","(27.907, 24.8)","(22.35, 24.0)"
worm,"(50.283, 6.9)","(50.343, 6.0)","(47.148, 6.5)","(48.124, 3.2)","(46.576, 3.9)","(46.275, 4.4)","(44.558, 6.2)","(43.032, 6.9)","(43.134, 5.7)"
yeast,"(22.537, 24.4)","(12.794, 20.6)","(17.163, 22.1)","(12.721, 20.5)","(17.164, 22.1)","(13.206, 21.2)","(10.628, 22.8)","(8.759, 18.4)","(8.812, 18.5)"


In [18]:
#sem ratioReduction
table_1_J48_without_ratioReduction = pd.DataFrame([
        [
            make_average(None, classifier = "J48", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-3', classifier = "J48", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-4', classifier = "J48", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-5', classifier = "J48", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-6', classifier = "J48", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-7', classifier = "J48", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-8', classifier = "J48", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-9', classifier = "J48", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-10', classifier = "J48", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
        ], 
        [
            make_average(None, classifier = "J48", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-3', classifier = "J48", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-4', classifier = "J48", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-5', classifier = "J48", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-6', classifier = "J48", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-7', classifier = "J48", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-8', classifier = "J48", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-9', classifier = "J48", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-10', classifier = "J48", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
        ],
        [
            make_average(None, classifier = "J48", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-3', classifier = "J48", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-4', classifier = "J48", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-5', classifier = "J48", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-6', classifier = "J48", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-7', classifier = "J48", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-8', classifier = "J48", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-9', classifier = "J48", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-10', classifier = "J48", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
        ],
        [
            make_average(None, classifier = "J48", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-3', classifier = "J48", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-4', classifier = "J48", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-5', classifier = "J48", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-6', classifier = "J48", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-7', classifier = "J48", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-8', classifier = "J48", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-9', classifier = "J48", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
            make_average('threshold-10', classifier = "J48", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None),
        ],
    ],
    columns=mux, 
    index=['fly', 'mouse', 'worm', 'yeast']
)

table_1_J48_without_ratioReduction

,Normal,T(3),T(4),T(5),T(6),T(7),T(8),T(9),T(10)
,Average,Average,Average,Average,Average,Average,Average,Average,Average
fly,"(37.754, 15.8)","(36.478, 16.2)","(36.805, 15.3)","(27.399, 20.5)","(22.278, 24.2)","(24.594, 22.0)","(24.594, 22.0)","(24.594, 22.0)","(0.0, 0.0)"
mouse,"(40.542, 24.1)","(42.071, 23.4)","(43.356, 23.9)","(41.55, 15.1)","(28.294, 25.0)","(30.147, 27.0)","(21.418, 22.6)","(21.418, 22.6)","(13.416, 21.6)"
worm,"(46.392, 10.7)","(45.719, 7.8)","(41.625, 6.7)","(35.976, 9.6)","(33.561, 9.7)","(30.139, 13.6)","(34.709, 9.4)","(32.164, 11.3)","(32.164, 11.3)"
yeast,"(25.843, 23.3)","(4.471, 14.1)","(13.416, 21.6)","(0.0, 0.0)","(0.0, 0.0)","(0.0, 0.0)","(0.0, 0.0)","(0.0, 0.0)","(0.0, 0.0)"


In [ ]:
#Com ratioReduction
table_1_KNN = pd.DataFrame([
        [
            make_average(None, classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average(None, classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-3', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-3', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-4', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-4', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-5', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-5', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-6', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-6', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-7', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-7', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-8', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-8', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-9', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-9', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-10', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-10', classifier = "KNN", organism = "fly", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
        ], 
        [
            make_average(None, classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average(None, classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-3', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-3', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-4', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-4', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-5', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-5', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-6', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-6', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-7', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-7', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-8', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-8', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-9', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-9', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-10', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-10', classifier = "KNN", organism = "mouse", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
        ],
        [
            make_average(None, classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average(None, classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-3', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-3', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-4', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-4', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-5', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-5', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-6', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-6', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-7', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-7', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-8', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-8', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-9', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-9', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-10', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-10', classifier = "KNN", organism = "worm", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
        ],
        [
            make_average(None, classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average(None, classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-3', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-3', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-4', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-4', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-5', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-5', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-6', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-6', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-7', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-7', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-8', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-8', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-9', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-9', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-10', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = "GMean", ratioRed = None), make_average('threshold-10', classifier = "KNN", organism = "yeast", agmoOrNot = 0, gMean = None, ratioRed = "ratioReduction", ),
        ],
    ],
    columns=mux, 
    index=['fly', 'mouse', 'worm', 'yeast']
)

table_1_KNN

In [ ]:
#Com ratioReduction
table_1_NB = pd.DataFrame([
        [
            make_average(None, classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average(None, classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-3', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-3', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-4', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-4', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-5', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-5', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-6', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-6', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-7', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-7', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-8', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-8', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-9', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-9', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-10', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-10', classifier = "NB", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
        ], 
        [
            make_average(None, classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average(None, classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-3', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-3', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-4', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-4', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-5', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-5', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-6', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-6', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-7', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-7', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-8', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-8', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-9', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-9', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-10', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-10', classifier = "NB", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
        ],
        [
            make_average(None, classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average(None, classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-3', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-3', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-4', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-4', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-5', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-5', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-6', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-6', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-7', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-7', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-8', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-8', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-9', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-9', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-10', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-10', classifier = "NB", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
        ],
        [
            make_average(None, classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average(None, classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-3', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-3', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-4', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-4', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-5', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-5', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-6', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-6', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-7', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-7', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-8', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-8', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-9', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-9', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-10', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-10', classifier = "NB", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
        ],
    ],
    columns=mux, 
    index=['fly', 'mouse', 'worm', 'yeast']
)

table_1_NB

In [ ]:
#Com ratioReduction
table_1_J48 = pd.DataFrame([
        [
            make_average(None, classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average(None, classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-3', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-3', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-4', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-4', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-5', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-5', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-6', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-6', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-7', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-7', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-8', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-8', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-9', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-9', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-10', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-10', classifier = "J48", organism = "fly", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
        ], 
        [
            make_average(None, classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average(None, classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-3', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-3', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-4', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-4', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-5', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-5', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-6', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-6', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-7', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-7', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-8', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-8', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-9', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-9', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-10', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-10', classifier = "J48", organism = "mouse", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
        ],
        [
            make_average(None, classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average(None, classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-3', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-3', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-4', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-4', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-5', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-5', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-6', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-6', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-7', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-7', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-8', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-8', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-9', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-9', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-10', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-10', classifier = "J48", organism = "worm", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
        ],
        [
            make_average(None, classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average(None, classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-3', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-3', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-4', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-4', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-5', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-5', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-6', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-6', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-7', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-7', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-8', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-8', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-9', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-9', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
            make_average('threshold-10', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = "GMean", ratioRed = None), make_average('threshold-10', classifier = "J48", organism = "yeast", agmoOrNot = 1, gMean = None, ratioRed = "ratioReduction", ),
        ],
    ],
    columns=mux, 
    index=['fly', 'mouse', 'worm', 'yeast']
)

table_1_J48

In [ ]:
table_1_J48_without_ratioReduction.to_csv("meansAndReducTables\\j48_without_selection.csv")
table_1_KNN_without_ratioReduction.to_csv("meansAndReducTables\\knn_without_selection.csv")
table_1_NB_without_ratioReduction.to_csv("meansAndReducTables\\nb_without_selection.csv")

In [11]:
table_1_J48.to_csv("meansAndReducTables\\j48.csv")
table_1_KNN.to_csv("meansAndReducTables\\knn.csv")
table_1_NB.to_csv("meansAndReducTables\\nb.csv")


In [12]:
stats.friedmanchisquare(results_classifiers['threshold-10']['KNN']['worm'][1]['GMean'],
                        results_classifiers['threshold-10']['KNN']['worm'][0]['GMean'],
                        results_classifiers['threshold-10']['NB']['worm'][1]['GMean'],
                        results_classifiers['threshold-10']['NB']['worm'][0]['GMean'],
                        results_classifiers['threshold-10']['J48']['worm'][1]['GMean'],
                        results_classifiers['threshold-10']['J48']['worm'][0]['GMean']
                        )

FriedmanchisquareResult(statistic=20.864553314121036, pvalue=0.000859201925330517)